# Data Set を作ります。

In [ ]:
import sys
import pathlib
import numpy
import PIL
import astropy.io.fits
import astropy.wcs
import tensorflow

sys.path.append('..')
from src.utils import data_utils

sample = data_utils.get_sample_table()

In [ ]:
def cut_obj(margin, obj_info):
    data_path = pathlib.Path('~/jupyter/spitzer_bubble/data/interim/gal').expanduser()
    dir_ = data_path/obj_info['directory']
    
    files =['r.fits', 'g.fits', 'b.fits']
    hdus = [astropy.io.fits.open(data_path/dir_/file)[0] for file in files]

    r_data = hdus[0].data
    g_data = hdus[1].data
    b_data = hdus[2].data    
    
    hdu = astropy.io.fits.open(dir_/'g.fits')[0]
    header = hdu.header
    w = astropy.wcs.WCS(header)

    l_min = obj_info['l'] - margin*obj_info['Rout']/60
    b_min = obj_info['b'] - margin*obj_info['Rout']/60
    l_max = obj_info['l'] + margin*obj_info['Rout']/60
    b_max = obj_info['b'] + margin*obj_info['Rout']/60
    x_pix_min, y_pix_min = w.all_world2pix(l_max, b_min, 0)
    x_pix_max, y_pix_max = w.all_world2pix(l_min, b_max, 0)

    R_pix = int(((x_pix_max - x_pix_min)/2 + (y_pix_max - y_pix_min)/2)/2)
    x_pix, y_pix = w.all_world2pix(obj_info['l'], obj_info['b'], 0)

    x_pix_min = max(0, int(numpy.round(x_pix)) - R_pix)
    x_pix_max = max(0, int(numpy.round(x_pix)) + R_pix)
    y_pix_min = max(0, int(numpy.round(y_pix)) - R_pix)
    y_pix_max = max(0, int(numpy.round(y_pix)) + R_pix)
    
    r_obj = r_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
    g_obj = g_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
    b_obj = b_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
    rgb_data = numpy.stack([r_obj, g_obj, b_obj], 2)
    rgb_data = numpy.flipud(rgb_data)
    
    return rgb_data

In [ ]:
obj = 'S7'
obj_info = sample.loc[obj]
rgb_data = cut_obj(3, obj_info)

In [ ]:
rgb_data = tensorflow.image.resize(
    images=numpy.array([rgb_data]),
    size=[256, 256],
    method='bilinear',
    preserve_aspect_ratio=True,
)

im = PIL.Image.fromarray(numpy.uint8(rgb_data[0]))
print(obj_info)
im

## 以下、試行錯誤の結果

## get_sample_table(汚い関数)で Data Set のテーブルを取得 & RGB FITS への Path 取得

In [ ]:
sample = data_utils.get_sample_table()
data_path = pathlib.Path('~/jupyter/spitzer_bubble/data/interim/gal').expanduser()

## 天体名を指定して RGB の HDU を取得 & data 取り出し

In [ ]:
obj = 'S10'
dir_ = sample.loc[:, 'directory'][obj]
files =['r.fits', 'g.fits', 'b.fits']
hdus = [astropy.io.fits.open(data_path/dir_/file)[0] for file in files]

r_data = hdus[0].data
g_data = hdus[1].data
b_data = hdus[2].data

# r_data = numpy.where(r_data>255, 255, r_data)
# r_data = numpy.where(r_data<0, 0, r_data)
# g_data = numpy.where(g_data>255, 255, g_data)
# g_data = numpy.where(g_data<0, 0, g_data)
# b_data = numpy.where(b_data>255, 255, b_data)
# b_data = numpy.where(b_data<0, 0, b_data)

#rgb_data = numpy.stack([r_data, g_data, b_data], 2)

## header と天体の座標から nparray の pixcel 範囲を計算

In [ ]:
margin = 3

header = hdus[0].header
w = astropy.wcs.WCS(header)

l_min = sample.loc[obj]['l'] - margin*sample.loc[obj]['Rout']/60
b_min = sample.loc[obj]['b'] - margin*sample.loc[obj]['Rout']/60
l_max = sample.loc[obj]['l'] + margin*sample.loc[obj]['Rout']/60
b_max = sample.loc[obj]['b'] + margin*sample.loc[obj]['Rout']/60
x_pix_min, y_pix_min = w.all_world2pix(l_max, b_min, 0)
x_pix_max, y_pix_max = w.all_world2pix(l_min, b_max, 0)

R_pix = int(((x_pix_max - x_pix_min)/2 + (y_pix_max - y_pix_min)/2)/2)
x_pix, y_pix = w.all_world2pix(sample.loc[obj]['l'], sample.loc[obj]['b'], 0)

x_pix_min = int(numpy.round(x_pix)) - R_pix
x_pix_max = int(numpy.round(x_pix)) + R_pix
y_pix_min = int(numpy.round(y_pix)) - R_pix
y_pix_max = int(numpy.round(y_pix)) + R_pix

if x_pix_min < 0:
    x_pix_min = 0
    pass
if y_pix_min < 0:
    y_pix_min = 0
    pass

In [ ]:
r_obj = r_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
g_obj = g_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
b_obj = b_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]

rgb_data = numpy.stack([r_obj, g_obj, b_obj], 2)
rgb_data = numpy.flipud(rgb_data)

In [ ]:
rgb_data = tensorflow.image.resize(
    images=numpy.array([rgb_data]),
    size=[256, 256],
    method='bilinear',
    preserve_aspect_ratio=True,
)
im = PIL.Image.fromarray(numpy.uint8(rgb_data[0]))
im

In [ ]:
def calc_pix(margin, obj_info):
    if type(obj_info) == pandas.core.series.Series:
        obj_info = dict(obj_info)
        pass
    path = pathlib.Path('~/jupyter/spitzer_bubble/data/interim/gal').expanduser()
    dir_ = path/obj_info['directory']
    
    hdu = astropy.io.fits.open(dir_/'g.fits')[0]
    header = hdu.header
    w = astropy.wcs.WCS(header)

    l_min = obj_info['l'] - margin*obj_info['Rout']/60
    b_min = obj_info['b'] - margin*obj_info['Rout']/60
    l_max = obj_info['l'] + margin*obj_info['Rout']/60
    b_max = obj_info['b'] + margin*obj_info['Rout']/60
    x_pix_min, y_pix_min = w.all_world2pix(l_max, b_min, 0)
    x_pix_max, y_pix_max = w.all_world2pix(l_min, b_max, 0)

    R_pix = int(((x_pix_max - x_pix_min)/2 + (y_pix_max - y_pix_min)/2)/2)
    x_pix, y_pix = w.all_world2pix(obj_info['l'], obj_info['b'], 0)

    x_pix_min = max(0, int(numpy.round(x_pix)) - R_pix)
    x_pix_max = max(0, int(numpy.round(x_pix)) + R_pix)
    y_pix_min = max(0, int(numpy.round(y_pix)) - R_pix)
    y_pix_max = max(0, int(numpy.round(y_pix)) + R_pix)
    
    return [x_pix_min, x_pix_max], [y_pix_min, y_pix_max]

In [ ]:
sample = data_utils.get_sample_table()

In [ ]:
obj = 'F30'
obj_info = sample.loc[obj]
rgb_data = cut_obj(3, obj_info)

In [ ]:
rgb_data = tensorflow.image.resize(
    images=numpy.array([rgb_data]),
    size=[256, 256],
    method='bilinear',
    preserve_aspect_ratio=True,
)

im = PIL.Image.fromarray(numpy.uint8(rgb_data[0]))
im